In [1]:
"""
This script prepares your Python environment by loading essential tools for handling environment variables, 
interacting with a Neo4j graph database, and utilizing a bespoke graph module, which could be sourced 
from either a local graph class or optionally from the trengx package.

import os: 
This line imports the built-in Python module os, which provides a portable way of using 
operating system dependent functionality, like reading or writing to the environment.

import sys and sys.path.append('../trengx'): 
These lines modify the system path that Python uses to look for modules. The sys module provides access 
to some variables used or maintained by the Python interpreter. In this case, it's used to add 
the parent directory's trengx subdirectory to the list of directories that Python will search when you import a module.

import graph: This line imports the graph module, which is expected to be found in the trengx directory 
that was just added to the system path. This module likely contains the Graph class that you use in your script. 
The specific functionality of this module depends on its implementation, but based on context, 
it likely provides tools for creating and manipulating a computational graph.
"""

# %pip install trengx
# from trengx import neo4j

import os
import sys
sys.path.append('../trengx')
import graph

In [2]:
# Retrieve secrets (confidential credentials) from environment variables stored in .env file
uri = os.getenv('NEO4J_URI') # Get the value of the uri variable
user = os.getenv('NEO4J_USER') # Get the value of the user variable
password= os.getenv('NEO4J_PASSWORD')  # Get the value of the password variable

# Create a Graph class instance using Graph class in neo4j_graph.py (module)
g = graph.Graph(uri, user, password)


In [3]:
g.delete_all()

True

In [1]:
query="""
    MATCH (in1)
    WHERE in1.uuid = $node_id
    SET in1.value = $value
    WITH in1
    MATCH path = (in1)-[r:num2op|op2num*0..]->()
    WHERE all(rel IN relationships(path) WHERE rel.trigger = true)
    WITH path
    ORDER BY length(path) DESC
    LIMIT 1
    WITH nodes(path) AS nodes
    UNWIND range(0, size(nodes)-2, 2) AS i
    WITH nodes[i] AS in1, nodes[i+1] AS op, nodes[i+2] AS out, nodes
    MATCH (in1)-[:num2op]->(op)-[:op2num]->(out)
    OPTIONAL MATCH (in2)-[:num2op]->(op)
    WHERE in1.uuid  <> in2.uuid
    WITH in1, op, out, in2, nodes
    SET out.value = 
    CASE 
        WHEN op.name = '+' THEN in1.value + in2.value
        WHEN op.name = '-' AND op.reverse = false THEN in1.value - in2.value
        WHEN op.name = '-' AND op.reverse = true THEN in2.value - in1.value
        WHEN op.name = '*' THEN in1.value * in2.value
        WHEN op.name = '/' AND op.reverse = false AND in2.value <> 0 THEN in1.value / in2.value
        WHEN op.name = '/' AND op.reverse = true AND in1.value <> 0 THEN in2.value / in1.value
        WHEN op.name = 'sqrt' AND in1.value >= 0 THEN sqrt(in1.value)
        WHEN op.name = 'abs' THEN abs(in1.value)
        WHEN op.name = 'exp' THEN exp(in1.value)
        WHEN op.name = 'log10' AND in1.value > 0 THEN log10(in1.value)
        WHEN op.name = 'log' AND in1.value > 0 THEN log(in1.value)
        WHEN op.name = 'sin' THEN sin(in1.value)
        WHEN op.name = 'cos' THEN cos(in1.value)
        WHEN op.name = 'tan' THEN tan(in1.value)
        WHEN op.name = 'ceil' THEN ceil(in1.value)
        WHEN op.name = 'floor' THEN floor(in1.value)
        WHEN op.name = 'round' THEN round(in1.value)
        WHEN op.name = 'sign' THEN sign(in1.value)
        WHEN op.name = 'store' THEN in1.value
        ELSE out.value
    END
    RETURN nodes
"""

In [5]:
# Define the parameters for the query.
parameters = {'node_id': '5798a9b1-d695-4c93-b081-442ce3784684', 'value': 10}

# Use the run_query method to execute the query and fetch the results.
results = g.run_query(query, parameters)

# Print the results.
print(results)

[]


In [6]:
e1_id = e1['id']
g.get_edge(e1_id)

NameError: name 'e1' is not defined

In [ ]:
g.get_node_property(a_id, 'name')

'a'

In [ ]:
g.update_node_properties (a_id, {'name': 'Kevin', 'value': 343}) 
node = g.get_node(a_id)  
node


{'id': '01b5f18e-eb5a-494c-ac46-57504c0f9e3a',
 'label': ['num2'],
 'properties': {'name': 'Kevin', 'value': 343}}

In [ ]:
g.update_edge_properties (e1_id, {'name': 'Kevin', 'value': 343}) 
edge = g.get_edge(e1_id)  
edge


{'id': '033fdca9-e75c-4788-ba3c-7e07fab69a31',
 'label': 'op',
 'out_id': '01b5f18e-eb5a-494c-ac46-57504c0f9e3a',
 'in_id': 'dc3620ac-07c8-4361-a7a0-5a073851e55d',
 'properties': {'name': 'Kevin', 'value': 343}}

In [ ]:
#g.delete_node(a_id, True)

In [ ]:
g.delete_edge(e1_id)

True